<a href="https://colab.research.google.com/github/yiyue02/5052_final_project/blob/main/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Netflix User Churn Prediction Project

## Objective
Predict whether a user will remain active (`is_active`) using recent watch patterns, engagement metrics, demographics, and subscription plan information. Identify the most influential drivers of churn.

## Dataset Overview
- **Source**: Netflix 2025: User Behavior Dataset (210K+ Records)
- **Target Variable**: `is_active` (binary) from users.csv
- **Features**: Demographics, subscription info, watch patterns, engagement metrics
- **Data Files**: users.csv, movies.csv, watch_history.csv, recommendation_logs.csv, search_logs.csv, reviews.csv

In [44]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%cd /content/ML_model

/content/ML_model


## 1. Data Loading and Initial Exploration

In [ ]:
# Load all datasets
users = pd.read_csv('archive/users.csv')
movies = pd.read_csv('archive/movies.csv')
watch_history = pd.read_csv('archive/watch_history.csv')
recommendation_logs = pd.read_csv('archive/recommendation_logs.csv')
search_logs = pd.read_csv('archive/search_logs.csv')
reviews = pd.read_csv('archive/reviews.csv')

print("Dataset Shapes:")
print(f"Users: {users.shape}")
print(f"Movies: {movies.shape}")
print(f"Watch History: {watch_history.shape}")
print(f"Recommendation Logs: {recommendation_logs.shape}")
print(f"Search Logs: {search_logs.shape}")
print(f"Reviews: {reviews.shape}")

# Display first few rows of users table (contains target variable)
print("\n" + "="*80)
print("Users Table Sample (Target: is_active):")
print("="*80)
users.head()

Dataset Shapes:
Users: (10300, 16)
Movies: (1040, 18)
Watch History: (105000, 12)
Recommendation Logs: (52000, 11)
Search Logs: (26500, 11)
Reviews: (15450, 12)

Users Table Sample (Target: is_active):


,user_id,email,first_name,last_name,age,gender,country,state_province,city,subscription_plan,subscription_start_date,is_active,monthly_spend,primary_device,household_size,created_at
0,user_00001,figueroajohn@example.org,Erica,Garza,43.0,Male,USA,Massachusetts,North Jefferyhaven,Basic,2024-04-08,True,36.06,Laptop,1.0,2023-04-01 14:40:50.540242
1,user_00002,blakeerik@example.com,Joshua,Bernard,38.0,Male,USA,Texas,North Noahstad,Premium+,2024-05-24,True,14.59,Desktop,2.0,2024-10-10 15:39:11.030515
2,user_00003,smiller@example.net,Barbara,Williams,32.0,Female,USA,Michigan,Traciebury,Standard,2023-09-22,False,11.71,Desktop,3.0,2024-06-29 14:27:49.560875
3,user_00004,mitchellclark@example.com,Chelsea,Ferguson,11.0,Male,USA,Ohio,South Noah,Standard,2024-08-21,True,28.56,Laptop,2.0,2023-04-11 01:01:59.614841
4,user_00005,richard13@example.net,Jason,Foster,21.0,Female,USA,Arizona,West Donald,Standard,2024-10-28,True,9.54,Desktop,6.0,2025-04-12 19:59:30.137806


In [ ]:
# Check data info and target distribution
print("Users Table Info:")
print(users.info())
print("\n" + "="*80)
print("Target Variable Distribution:")
print("="*80)
target_dist = users['is_active'].value_counts()
print(target_dist)
print(f"\nChurn Rate: {(1 - target_dist[True] / len(users)) * 100:.2f}%")

# Visualize target distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
users['is_active'].value_counts().plot(kind='bar', ax=axes[0], color=['#ff6b6b', '#4ecdc4'])
axes[0].set_title('User Activity Status Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Is Active')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Inactive', 'Active'], rotation=0)

# Pie chart
colors = ['#ff6b6b', '#4ecdc4']
axes[1].pie(target_dist, labels=['Active', 'Inactive'], autopct='%1.1f%%',
            colors=colors, startangle=90)
axes[1].set_title('User Activity Proportion', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 2. Data Quality Assessment

In [ ]:
# Check for missing values and duplicates
print("="*80)
print("DATA QUALITY ASSESSMENT")
print("="*80)

print("\n1. MISSING VALUES:")
print("-" * 80)
missing_users = users.isnull().sum()
missing_pct = (missing_users / len(users) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing_users, 'Percentage': missing_pct})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
print(missing_df)

print("\n2. DUPLICATE RECORDS:")
print("-" * 80)
print(f"Users duplicates: {users.duplicated().sum()} ({users.duplicated().sum() / len(users) * 100:.2f}%)")
print(f"Watch history duplicates: {watch_history.duplicated().sum()} ({watch_history.duplicated().sum() / len(watch_history) * 100:.2f}%)")

print("\n3. DATA TYPES:")
print("-" * 80)
print(users.dtypes)

In [ ]:
# Visualize missing data patterns
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Missing values heatmap
ax = axes[0]
missing_matrix = users.isnull()
if missing_matrix.sum().sum() > 0:
    sns.heatmap(missing_matrix, cbar=True, yticklabels=False, cmap='YlOrRd', ax=ax)
    ax.set_title('Missing Values Pattern (Yellow = Missing)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Features')
else:
    ax.text(0.5, 0.5, 'No Missing Values', ha='center', va='center', fontsize=14)
    ax.set_title('Missing Values Pattern', fontsize=12, fontweight='bold')

# Missing percentage by column
ax = axes[1]
if missing_df.shape[0] > 0:
    ax.barh(missing_df.index, missing_df['Percentage'], color='coral', alpha=0.7)
    ax.set_xlabel('Missing Percentage (%)', fontsize=11)
    ax.set_title('Missing Values by Feature', fontsize=12, fontweight='bold')
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=0.3)
else:
    ax.text(0.5, 0.5, 'No Missing Values', ha='center', va='center', fontsize=14)
    ax.set_title('Missing Values by Feature', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

### Handling Duplicates Strategy

**Order of Operations**: Handle duplicates BEFORE missing value imputation to avoid artificially inflating patterns.

Duplicate records can arise from:
- **Data collection errors**: Same user recorded multiple times
- **System issues**: Double recording of transactions
- **Natural duplicates**: Multiple people with same characteristics

**Strategy**:
- Identify duplicates based on unique identifier (user_id)
- Keep first occurrence (most recent or complete record)
- Remove exact duplicates across all columns

In [ ]:
# STEP 1: Handle duplicates FIRST
print("="*80)
print("STEP 1: DUPLICATE HANDLING")
print("="*80)

# Check for duplicates based on user_id (should be unique)
print("\n1. USER_ID DUPLICATES:")
user_id_duplicates = users[users.duplicated(subset=['user_id'], keep=False)]
print(f"   Records with duplicate user_id: {len(user_id_duplicates)}")

users_clean = users.copy()

if len(user_id_duplicates) > 0:
    print(f"   Unique users with duplicates: {user_id_duplicates['user_id'].nunique()}")
    # Keep first occurrence
    users_clean = users_clean.drop_duplicates(subset=['user_id'], keep='first')
    print(f"   Strategy: Keep first occurrence")
    print(f"   Removed: {len(users) - len(users_clean)} records")
else:
    print(f"   No user_id duplicates found!")

# Check for exact duplicates (all columns identical)
print("\n2. EXACT DUPLICATES (all columns):")
exact_duplicates = users_clean.duplicated()
print(f"   Exact duplicate records: {exact_duplicates.sum()}")

if exact_duplicates.sum() > 0:
    users_clean = users_clean.drop_duplicates(keep='first')
    print(f"   Strategy: Keep first occurrence")
    print(f"   Removed: {exact_duplicates.sum()} records")
else:
    print(f"   No exact duplicates found!")

# Handle duplicates in watch_history
print("\n3. WATCH HISTORY DUPLICATES:")
watch_dup = watch_history.duplicated()
print(f"   Exact duplicate sessions: {watch_dup.sum()} ({watch_dup.sum()/len(watch_history)*100:.2f}%)")

if watch_dup.sum() > 0:
    watch_history_clean = watch_history.drop_duplicates(keep='first')
    print(f"   Strategy: Remove exact duplicates, keep first")
    print(f"   Removed: {watch_dup.sum()} records")
    watch_history = watch_history_clean

print(f"\n{'='*80}")
print(f"Users before:        {len(users)}")
print(f"Users after:         {len(users_clean)}")
print(f"Records removed:     {len(users) - len(users_clean)}")
print(f"Data retention rate: {len(users_clean)/len(users)*100:.2f}%")
print(f"{'='*80}")

# Update users dataframe for next step
users = users_clean

print("\n" + "="*80)
print("STEP 2: MISSING VALUE IMPUTATION STRATEGY")
print("="*80)

# 1. Age - DO NOT CHANGE (keep missing as is or will be handled later)
if users['age'].isnull().sum() > 0:
    print(f"\n1. AGE:")
    print(f"   Missing: {users['age'].isnull().sum()} ({users['age'].isnull().sum()/len(users)*100:.2f}%)")
    print(f"   Strategy: NO CHANGE - Keep missing values as is")

# 2. Gender - categorical, use 'Prefer not to say'
if users['gender'].isnull().sum() > 0:
    print(f"\n2. GENDER:")
    print(f"   Missing: {users['gender'].isnull().sum()} ({users['gender'].isnull().sum()/len(users)*100:.2f}%)")
    print(f"   Strategy: Fill with 'Prefer not to say' category")
    users['gender'] = users['gender'].fillna('Prefer not to say')

# 3. Monthly spend - 0 for inactive accounts (assuming free users or inactive)
if users['monthly_spend'].isnull().sum() > 0:
    print(f"\n3. MONTHLY SPEND:")
    print(f"   Missing: {users['monthly_spend'].isnull().sum()} ({users['monthly_spend'].isnull().sum()/len(users)*100:.2f}%)")
    print(f"   Strategy: Fill with 0 (assuming free users or inactive accounts)")
    users['monthly_spend'] = users['monthly_spend'].fillna(0)

# 4. Household size - use median by country
if users['household_size'].isnull().sum() > 0:
    print(f"\n4. HOUSEHOLD SIZE:")
    print(f"   Missing: {users['household_size'].isnull().sum()} ({users['household_size'].isnull().sum()/len(users)*100:.2f}%)")
    print(f"   Strategy: Fill with median by country")

    # Calculate median household size by country
    country_medians = users.groupby('country')['household_size'].median()
    print(f"   Countries with data: {len(country_medians)}")

    # Fill missing values with country-specific median (fallback to overall median if country median is NaN)
    users['household_size'] = users.apply(
        lambda row: country_medians.get(row['country'], users['household_size'].median())
        if pd.isnull(row['household_size']) else row['household_size'],
        axis=1
    )
    print(f"   Applied country-specific medians")

print(f"\n{'='*80}")
print(f"IMPUTATION COMPLETE")
print(f"{'='*80}")
print(f"Total missing values after imputation: {users.isnull().sum().sum()}")
if users.isnull().sum().sum() > 0:
    print(f"\nRemaining missing values by column:")
    print(users.isnull().sum()[users.isnull().sum() > 0])

### Handling Outliers

Outliers can significantly impact model performance. Common approaches:
- **Capping/Clipping**: Set min/max thresholds (e.g., age 18-100)
- **IQR Method**: Remove/cap values beyond Q1 - 1.5×IQR or Q3 + 1.5×IQR
- **Z-score**: Remove values with |z-score| > 3
- **Domain knowledge**: Use business logic (e.g., monthly spend caps)

In [ ]:
# Identify and handle outliers
print("="*80)
print("OUTLIER DETECTION AND TREATMENT")
print("="*80)

# Function to detect outliers using IQR method
def detect_outliers_iqr(data, column, multiplier=1.5):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# 1. Age outliers
print("\n1. AGE OUTLIERS:")
age_outliers, age_lower, age_upper = detect_outliers_iqr(users, 'age')
print(f"   Outliers detected: {len(age_outliers)} ({len(age_outliers)/len(users)*100:.2f}%)")
print(f"   IQR bounds: [{age_lower:.1f}, {age_upper:.1f}]")
print(f"   Actual range: [{users['age'].min():.1f}, {users['age'].max():.1f}]")
print(f"   Strategy: Cap at [18, 100] based on domain knowledge")
users['age'] = users['age'].clip(18, 100)
print(f"   After capping: [{users['age'].min():.1f}, {users['age'].max():.1f}]")

# 2. Monthly spend outliers
print("\n2. MONTHLY SPEND OUTLIERS:")
spend_outliers, spend_lower, spend_upper = detect_outliers_iqr(users, 'monthly_spend', multiplier=3)
print(f"   Outliers detected: {len(spend_outliers)} ({len(spend_outliers)/len(users)*100:.2f}%)")
print(f"   IQR bounds (3×IQR): [${spend_lower:.2f}, ${spend_upper:.2f}]")
print(f"   Actual range: [${users['monthly_spend'].min():.2f}, ${users['monthly_spend'].max():.2f}]")
print(f"   Strategy: Cap at 3×IQR bounds (conservative approach)")
users['monthly_spend'] = users['monthly_spend'].clip(spend_lower, spend_upper)
print(f"   After capping: [${users['monthly_spend'].min():.2f}, ${users['monthly_spend'].max():.2f}]")

# 3. Household size outliers
print("\n3. HOUSEHOLD SIZE OUTLIERS:")
household_outliers, household_lower, household_upper = detect_outliers_iqr(users, 'household_size')
print(f"   Outliers detected: {len(household_outliers)} ({len(household_outliers)/len(users)*100:.2f}%)")
print(f"   IQR bounds: [{household_lower:.1f}, {household_upper:.1f}]")
print(f"   Actual range: [{users['household_size'].min():.0f}, {users['household_size'].max():.0f}]")
print(f"   Strategy: Cap at reasonable limit [1, 10]")
users['household_size'] = users['household_size'].clip(1, 10)
print(f"   After capping: [{users['household_size'].min():.0f}, {users['household_size'].max():.0f}]")

print(f"\n{'='*80}")
print(f"OUTLIER TREATMENT COMPLETE")
print(f"{'='*80}")

In [ ]:
# Visualize data distributions before and after cleaning
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Age distribution
ax = axes[0, 0]
ax.hist(users['age'], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('Age', fontsize=11)
ax.set_ylabel('Frequency', fontsize=11)
ax.set_title('Age Distribution (After Cleaning)', fontsize=12, fontweight='bold')
ax.axvline(users['age'].median(), color='red', linestyle='--', label=f"Median: {users['age'].median():.1f}")
ax.legend()
ax.grid(alpha=0.3)

# Monthly spend distribution
ax = axes[0, 1]
ax.hist(users['monthly_spend'], bins=30, color='coral', alpha=0.7, edgecolor='black')
ax.set_xlabel('Monthly Spend ($)', fontsize=11)
ax.set_ylabel('Frequency', fontsize=11)
ax.set_title('Monthly Spend Distribution (After Cleaning)', fontsize=12, fontweight='bold')
ax.axvline(users['monthly_spend'].median(), color='red', linestyle='--',
          label=f"Median: ${users['monthly_spend'].median():.2f}")
ax.legend()
ax.grid(alpha=0.3)

# Household size distribution
ax = axes[0, 2]
household_counts = users['household_size'].value_counts().sort_index()
ax.bar(household_counts.index, household_counts.values, color='lightgreen', alpha=0.7, edgecolor='black')
ax.set_xlabel('Household Size', fontsize=11)
ax.set_ylabel('Count', fontsize=11)
ax.set_title('Household Size Distribution (After Cleaning)', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Gender distribution
ax = axes[1, 0]
gender_counts = users['gender'].value_counts()
ax.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=90,
       colors=plt.cm.Set3(range(len(gender_counts))))
ax.set_title('Gender Distribution', fontsize=12, fontweight='bold')

# Subscription plan distribution
ax = axes[1, 1]
plan_counts = users['subscription_plan'].value_counts()
ax.bar(range(len(plan_counts)), plan_counts.values, color='purple', alpha=0.6, edgecolor='black')
ax.set_xticks(range(len(plan_counts)))
ax.set_xticklabels(plan_counts.index, rotation=45, ha='right')
ax.set_xlabel('Subscription Plan', fontsize=11)
ax.set_ylabel('Count', fontsize=11)
ax.set_title('Subscription Plan Distribution', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Active vs Inactive users
ax = axes[1, 2]
active_counts = users['is_active'].value_counts()
colors_active = ['#ff6b6b', '#4ecdc4']
bars = ax.bar(['Inactive', 'Active'], [active_counts[False], active_counts[True]],
              color=colors_active, alpha=0.7, edgecolor='black')
ax.set_ylabel('Count', fontsize=11)
ax.set_title('User Activity Status (Target)', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}\n({height/len(users)*100:.1f}%)',
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("="*80)
print("DATA QUALITY SUMMARY AFTER CLEANING")
print("="*80)
print(f"Total users: {len(users)}")
print(f"Total features: {users.shape[1]}")
print(f"Missing values: {users.isnull().sum().sum()}")
print(f"Duplicate records: {users.duplicated().sum()}")
print(f"\nData is ready for feature engineering!")

## 3. Feature Engineering - Create Engagement Metrics

In [ ]:
# Create user-level engagement features from watch history
print("Creating engagement features from watch history...")

# Aggregate watch history per user
watch_features = watch_history.groupby('user_id').agg({
    'session_id': 'count',  # Total sessions
    'watch_duration_minutes': ['sum', 'mean', 'std'],  # Watch duration stats
    'progress_percentage': ['mean', 'std'],  # Completion stats
    'is_download': 'sum',  # Downloads count
    'user_rating': 'count'  # Number of ratings given
}).reset_index()

# Flatten column names
watch_features.columns = ['user_id', 'total_sessions', 'total_watch_minutes',
                         'avg_watch_minutes', 'std_watch_minutes',
                         'avg_progress', 'std_progress', 'total_downloads', 'total_ratings']

# Fill NaN std values with 0 (for users with single session)
watch_features['std_watch_minutes'] = watch_features['std_watch_minutes'].fillna(0)
watch_features['std_progress'] = watch_features['std_progress'].fillna(0)

# Count unique actions per user
action_counts = watch_history.groupby(['user_id', 'action']).size().unstack(fill_value=0).reset_index()
action_counts.columns = ['user_id'] + [f'action_{col}' for col in action_counts.columns[1:]]

# Merge action counts
watch_features = watch_features.merge(action_counts, on='user_id', how='left')

print(f"Watch features shape: {watch_features.shape}")
print("\nSample watch features:")
watch_features.head()

In [ ]:
# Create recommendation engagement features
print("Creating recommendation features...")

rec_features = recommendation_logs.groupby('user_id').agg({
    'recommendation_id': 'count',  # Total recommendations
    'was_clicked': 'sum'  # Total clicks (was_clicked is boolean, sum gives count of True)
}).reset_index()

rec_features.columns = ['user_id', 'total_recommendations', 'total_clicks']

# Calculate click-through rate (avoid division by zero)
rec_features['click_through_rate'] = rec_features.apply(
    lambda row: row['total_clicks'] / row['total_recommendations'] if row['total_recommendations'] > 0 else 0,
    axis=1
)

print(f"Recommendation features shape: {rec_features.shape}")
print(f"Sample recommendation features:")
print(rec_features.head())

# Create search engagement features
print("\nCreating search features...")

search_features = search_logs.groupby('user_id').agg({
    'search_id': 'count',  # Total searches
    'results_returned': 'mean',  # Average results
    'search_query': lambda x: x.str.len().mean()  # Average query length
}).reset_index()

search_features.columns = ['user_id', 'total_searches', 'avg_results', 'avg_query_length']

print(f"Search features shape: {search_features.shape}")
print(f"Sample search features:")
print(search_features.head())

# Create review engagement features
print("\nCreating review features...")

review_features = reviews.groupby('user_id').agg({
    'review_id': 'count',  # Total reviews
    'rating': 'mean',  # Average rating given
    'helpful_votes': 'sum'  # Total helpful votes
}).reset_index()

review_features.columns = ['user_id', 'total_reviews', 'avg_rating_given', 'total_helpful_votes']

print(f"Review features shape: {review_features.shape}")
print(f"Sample review features:")
print(review_features.head())

## 4. Merge Features and Data Preprocessing

### Feature Summary and Visualization

Let's examine all the features we've created from different data sources and visualize their distributions and statistics.

In [ ]:
# Feature Summary Tables
print("="*80)
print("FEATURE ENGINEERING SUMMARY")
print("="*80)

# 1. Watch History Features
print("\n1. WATCH HISTORY FEATURES")
print("-" * 80)
print(f"Number of features: {len(watch_features.columns) - 1}")  # excluding user_id
print(f"Users with watch history: {len(watch_features)}")
print("\nFeature list:")
for col in watch_features.columns[1:]:
    print(f"  - {col}")

print("\nWatch Features Statistics:")
watch_stats = watch_features.drop('user_id', axis=1).describe().round(2)
print(watch_stats)

# 2. Recommendation Features
print("\n" + "="*80)
print("2. RECOMMENDATION FEATURES")
print("-" * 80)
print(f"Number of features: {len(rec_features.columns) - 1}")
print(f"Users with recommendations: {len(rec_features)}")
print("\nFeature list:")
for col in rec_features.columns[1:]:
    print(f"  - {col}")

print("\nRecommendation Features Statistics:")
rec_stats = rec_features.drop('user_id', axis=1).describe().round(2)
print(rec_stats)

# 3. Search Features
print("\n" + "="*80)
print("3. SEARCH FEATURES")
print("-" * 80)
print(f"Number of features: {len(search_features.columns) - 1}")
print(f"Users with search history: {len(search_features)}")
print("\nFeature list:")
for col in search_features.columns[1:]:
    print(f"  - {col}")

print("\nSearch Features Statistics:")
search_stats = search_features.drop('user_id', axis=1).describe().round(2)
print(search_stats)

# 4. Review Features
print("\n" + "="*80)
print("4. REVIEW FEATURES")
print("-" * 80)
print(f"Number of features: {len(review_features.columns) - 1}")
print(f"Users with reviews: {len(review_features)}")
print("\nFeature list:")
for col in review_features.columns[1:]:
    print(f"  - {col}")

print("\nReview Features Statistics:")
review_stats = review_features.drop('user_id', axis=1).describe().round(2)
print(review_stats)

# Overall summary
print("\n" + "="*80)
print("OVERALL FEATURE SUMMARY")
print("="*80)
total_features = (len(watch_features.columns) - 1 + len(rec_features.columns) - 1 +
                  len(search_features.columns) - 1 + len(review_features.columns) - 1)
print(f"Total engineered features: {total_features}")
print(f"  - Watch history: {len(watch_features.columns) - 1}")
print(f"  - Recommendations: {len(rec_features.columns) - 1}")
print(f"  - Search: {len(search_features.columns) - 1}")
print(f"  - Reviews: {len(review_features.columns) - 1}")

In [ ]:
# Visualize feature distributions
fig, axes = plt.subplots(3, 4, figsize=(18, 14))
fig.suptitle('Engineered Features Distribution', fontsize=16, fontweight='bold', y=0.995)

# Watch History Features (top 4)
ax = axes[0, 0]
ax.hist(watch_features['total_sessions'], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('Total Sessions', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Watch: Total Sessions', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

ax = axes[0, 1]
ax.hist(watch_features['total_watch_minutes'], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('Total Watch Minutes', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Watch: Total Minutes', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

ax = axes[0, 2]
ax.hist(watch_features['avg_progress'], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('Average Progress (%)', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Watch: Avg Progress', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

ax = axes[0, 3]
ax.hist(watch_features['total_downloads'], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('Total Downloads', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Watch: Downloads', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

# Recommendation Features
ax = axes[1, 0]
ax.hist(rec_features['total_recommendations'], bins=30, color='coral', alpha=0.7, edgecolor='black')
ax.set_xlabel('Total Recommendations', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Rec: Total Recommendations', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

ax = axes[1, 1]
ax.hist(rec_features['total_clicks'], bins=30, color='coral', alpha=0.7, edgecolor='black')
ax.set_xlabel('Total Clicks', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Rec: Total Clicks', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

ax = axes[1, 2]
ax.hist(rec_features['click_through_rate'], bins=30, color='coral', alpha=0.7, edgecolor='black')
ax.set_xlabel('Click-Through Rate', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Rec: Click-Through Rate', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

# Search Features
ax = axes[1, 3]
ax.hist(search_features['total_searches'], bins=30, color='lightgreen', alpha=0.7, edgecolor='black')
ax.set_xlabel('Total Searches', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Search: Total Searches', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

ax = axes[2, 0]
ax.hist(search_features['avg_results'], bins=30, color='lightgreen', alpha=0.7, edgecolor='black')
ax.set_xlabel('Avg Results', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Search: Avg Results', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

ax = axes[2, 1]
ax.hist(search_features['avg_query_length'], bins=30, color='lightgreen', alpha=0.7, edgecolor='black')
ax.set_xlabel('Avg Query Length', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Search: Avg Query Length', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

# Review Features
ax = axes[2, 2]
ax.hist(review_features['total_reviews'], bins=30, color='mediumpurple', alpha=0.7, edgecolor='black')
ax.set_xlabel('Total Reviews', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Review: Total Reviews', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

ax = axes[2, 3]
ax.hist(review_features['avg_rating_given'], bins=30, color='mediumpurple', alpha=0.7, edgecolor='black')
ax.set_xlabel('Avg Rating Given', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Review: Avg Rating', fontsize=11, fontweight='bold')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Feature correlation heatmap (top features)
print("="*80)
print("FEATURE CORRELATION ANALYSIS")
print("="*80)

# Select key features for correlation analysis
key_features = pd.DataFrame()
key_features['total_sessions'] = watch_features['total_sessions']
key_features['total_watch_minutes'] = watch_features['total_watch_minutes']
key_features['avg_progress'] = watch_features['avg_progress']
key_features['total_downloads'] = watch_features['total_downloads']
key_features['total_recommendations'] = rec_features['total_recommendations']
key_features['click_through_rate'] = rec_features['click_through_rate']
key_features['total_searches'] = search_features['total_searches']
key_features['total_reviews'] = review_features['total_reviews']
key_features['avg_rating_given'] = review_features['avg_rating_given']

# Calculate correlation matrix
correlation_matrix = key_features.corr()

# Plot correlation heatmap
fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Feature Correlation Heatmap (Key Features)', fontsize=14, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print top correlations
print("\nTop Positive Correlations:")
print("-" * 80)
# Get upper triangle of correlation matrix
upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)
# Stack and sort
correlations = upper_triangle.stack().sort_values(ascending=False)
print(correlations.head(10))

print("\nTop Negative Correlations:")
print("-" * 80)
print(correlations.tail(10))

In [ ]:
# Feature comparison by user activity status
print("="*80)
print("FEATURE COMPARISON: ACTIVE vs INACTIVE USERS")
print("="*80)

# Merge features with user activity status for comparison
feature_comparison = users[['user_id', 'is_active']].copy()
feature_comparison = feature_comparison.merge(watch_features, on='user_id', how='left')
feature_comparison = feature_comparison.merge(rec_features, on='user_id', how='left')
feature_comparison = feature_comparison.merge(search_features, on='user_id', how='left')
feature_comparison = feature_comparison.merge(review_features, on='user_id', how='left')

# Fill NaN with 0 for users with no activity
feature_comparison = feature_comparison.fillna(0)

# Compare key metrics between active and inactive users
print("\nKey Metrics Comparison:")
print("-" * 80)

comparison_metrics = ['total_sessions', 'total_watch_minutes', 'avg_progress',
                     'total_recommendations', 'click_through_rate',
                     'total_searches', 'total_reviews', 'avg_rating_given']

comparison_table = []
for metric in comparison_metrics:
    if metric in feature_comparison.columns:
        active_mean = feature_comparison[feature_comparison['is_active'] == True][metric].mean()
        inactive_mean = feature_comparison[feature_comparison['is_active'] == False][metric].mean()
        difference = active_mean - inactive_mean
        pct_diff = (difference / inactive_mean * 100) if inactive_mean != 0 else 0

        comparison_table.append({
            'Feature': metric,
            'Active (mean)': f'{active_mean:.2f}',
            'Inactive (mean)': f'{inactive_mean:.2f}',
            'Difference': f'{difference:.2f}',
            'Change (%)': f'{pct_diff:.1f}%'
        })

comparison_df = pd.DataFrame(comparison_table)
print(comparison_df.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(2, 4, figsize=(18, 8))
fig.suptitle('Feature Comparison: Active vs Inactive Users', fontsize=16, fontweight='bold')

for idx, metric in enumerate(comparison_metrics[:8]):
    row = idx // 4
    col = idx % 4
    ax = axes[row, col]

    if metric in feature_comparison.columns:
        active_data = feature_comparison[feature_comparison['is_active'] == True][metric]
        inactive_data = feature_comparison[feature_comparison['is_active'] == False][metric]

        # Box plot
        box_data = [active_data, inactive_data]
        bp = ax.boxplot(box_data, labels=['Active', 'Inactive'], patch_artist=True)
        bp['boxes'][0].set_facecolor('#4ecdc4')
        bp['boxes'][1].set_facecolor('#ff6b6b')

        ax.set_ylabel(metric.replace('_', ' ').title(), fontsize=9)
        ax.set_title(metric.replace('_', ' ').title(), fontsize=10, fontweight='bold')
        ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Merge all features with users table
print("Merging all features...")

df = users.copy()
df = df.merge(watch_features, on='user_id', how='left')
df = df.merge(rec_features, on='user_id', how='left')
df = df.merge(search_features, on='user_id', how='left')
df = df.merge(review_features, on='user_id', how='left')

print(f"Merged dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

# Handle missing values from merges (users with no activity in certain tables)
print("\nFilling missing values from left joins with 0...")
activity_columns = watch_features.columns.tolist()[1:] + rec_features.columns.tolist()[1:] + \
                   search_features.columns.tolist()[1:] + review_features.columns.tolist()[1:]

for col in activity_columns:
    if col in df.columns:
        df[col] = df[col].fillna(0)

print(f"\nMissing values after filling activity metrics:")
print(df.isnull().sum().sum())

In [ ]:
# Prepare features for modeling
print("="*80)
print("FEATURE PREPARATION FOR MODELING")
print("="*80)

# Select features for modeling
# Exclude: user_id, email, names, dates, text fields
exclude_cols = ['user_id', 'email', 'first_name', 'last_name', 'subscription_start_date', 'created_at']

# Separate target variable
y = df['is_active'].astype(int)
X = df.drop(columns=exclude_cols + ['is_active'])

print(f"\nTarget variable shape: {y.shape}")
print(f"Features shape: {X.shape}")
print(f"\nFeatures to be used ({len(X.columns)}):")
print(X.columns.tolist())

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nCategorical features ({len(categorical_cols)}): {categorical_cols}")
print(f"Numerical features ({len(numerical_cols)}): {len(numerical_cols)}")

In [ ]:
# Check categorical values before encoding
print("="*80)
print("CATEGORICAL FEATURE VALUES CHECK")
print("="*80)

for col in categorical_cols:
    print(f"\n{col.upper()}:")
    print("-" * 80)
    value_counts = X[col].value_counts()
    print(f"Total unique values: {len(value_counts)}")
    print(f"Value distribution:")
    print(value_counts)


# Check for any unusual values
print(f"\nGender value lengths:")
for val in X['gender'].unique():
    print(f"  '{val}': length = {len(str(val))}")

In [ ]:
# Encode categorical variables
print("Encoding categorical variables...")

label_encoders = {}
X_encoded = X.copy()

for col in categorical_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    label_encoders[col] = le
    print(f"  {col}: {len(le.classes_)} unique values")

print(f"\nEncoded features shape: {X_encoded.shape}")
print("\nSample of encoded data:")
X_encoded.head()

In [ ]:
# Check for NaN values before train-test split
print("="*80)
print("DATA VALIDATION")
print("="*80)
print(f"\nChecking for NaN values in encoded features...")
nan_count = X_encoded.isnull().sum().sum()
print(f"Total NaN values: {nan_count}")

if nan_count > 0:
    print("\nNaN values found per column:")
    nan_cols = X_encoded.isnull().sum()[X_encoded.isnull().sum() > 0]
    print(nan_cols)
    print("\nFilling NaN values with 0 (after encoding, NaN typically means missing category)...")
    X_encoded = X_encoded.fillna(0)
    print("NaN values filled successfully.")
else:
    print("No NaN values found. Data is clean.")

print(f"\n{'='*80}")

# Train-test split
print("TRAIN-TEST SPLIT")
print("="*80)

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nTraining set target distribution:")
print(y_train.value_counts())
print(f"\nTest set target distribution:")
print(y_test.value_counts())

# Store number of features for mtry calculations
n_features = X_train.shape[1]
print(f"\n{'='*80}")
print(f"Total number of features (p): {n_features}")
print(f"  - sqrt(p) = {int(np.sqrt(n_features))}")
print(f"  - p/3 ≈ {int(n_features/3)}")
print(f"{'='*80}")

## 5. Random Forest with Different m Values

We'll train Random Forest models with three different values of m (max_features):
1. **m = sqrt(p)**: Default for classification (most common)
2. **m = p/2 = 15**: Half of the features at each split
3. **m = p = 30** (Bagging): Uses all features at each split

In [ ]:
# Define different m values to test
m_values = {
    'm = sqrt(p)': int(np.sqrt(n_features)),
    'm = p/2': n_features // 2,  # p/2 = 15
    'm = p (Bagging)': n_features  # p = 30
}

# Store results
rf_results = {}

print("="*80)
print("TRAINING RANDOM FOREST MODELS WITH DIFFERENT m VALUES")
print("="*80)

for name, m_val in m_values.items():
    print(f"\n{'='*80}")
    print(f"Training Random Forest with {name} = {m_val}")
    print(f"{'='*80}")

    # Train Random Forest
    rf_model = RandomForestClassifier(
        n_estimators=500,
        max_features=m_val,
        random_state=42,
        n_jobs=-1,
        min_samples_split=5,
        min_samples_leaf=2
    )

    rf_model.fit(X_train, y_train)

    # Predictions
    y_train_pred = rf_model.predict(X_train)
    y_test_pred = rf_model.predict(X_test)
    y_test_proba = rf_model.predict_proba(X_test)[:, 1]

    # Calculate metrics
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    train_error = 1 - train_accuracy
    test_error = 1 - test_accuracy
    auc_score = roc_auc_score(y_test, y_test_proba)

    # Store results
    rf_results[name] = {
        'model': rf_model,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'train_error': train_error,
        'test_error': test_error,
        'auc': auc_score,
        'predictions': y_test_pred,
        'probabilities': y_test_proba,
        'confusion_matrix': confusion_matrix(y_test, y_test_pred)
    }

    print(f"\nResults:")
    print(f"  Training Accuracy: {train_accuracy:.4f}")
    print(f"  Training Error:    {train_error:.4f}")
    print(f"  Test Accuracy:     {test_accuracy:.4f}")
    print(f"  Test Error:        {test_error:.4f}")
    print(f"  AUC Score:         {auc_score:.4f}")

print(f"\n{'='*80}")
print("All Random Forest models trained successfully!")
print(f"{'='*80}")

In [ ]:
# Compare Random Forest models
comparison_df = pd.DataFrame({
    'Model': list(rf_results.keys()),
    'm_value': [m_values[k] for k in rf_results.keys()],
    'Train_Accuracy': [rf_results[k]['train_accuracy'] for k in rf_results.keys()],
    'Test_Accuracy': [rf_results[k]['test_accuracy'] for k in rf_results.keys()],
    'Train_Error': [rf_results[k]['train_error'] for k in rf_results.keys()],
    'Test_Error': [rf_results[k]['test_error'] for k in rf_results.keys()],
    'AUC': [rf_results[k]['auc'] for k in rf_results.keys()]
})

print("="*80)
print("RANDOM FOREST MODEL COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))

# Find best model
best_model_name = comparison_df.loc[comparison_df['Test_Error'].idxmin(), 'Model']
print(f"\n{'='*80}")
print(f"Best Random Forest Model: {best_model_name}")
print(f"  Test Error: {rf_results[best_model_name]['test_error']:.4f}")
print(f"  Test Accuracy: {rf_results[best_model_name]['test_accuracy']:.4f}")
print(f"  AUC: {rf_results[best_model_name]['auc']:.4f}")
print(f"{'='*80}")

In [ ]:
# Feature Importance - Random Forest Models Comparison
print("="*80)
print("FEATURE IMPORTANCE COMPARISON ACROSS RANDOM FOREST MODELS")
print("="*80)

# Get feature importance from all Random Forest models
model_names = list(rf_results.keys())
n_models = len(model_names)

# Create subplot layout: 1x3 for 3 models
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
fig.suptitle('Random Forest: Feature Importance Comparison for Different m Values',
             fontsize=16, fontweight='bold', y=1.02)

colors = ['steelblue', 'forestgreen', 'coral']

for idx, name in enumerate(model_names):
    ax = axes[idx]
    model = rf_results[name]['model']

    # Get feature importance
    importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)

    # Plot top 15 features
    top_15 = importance_df.head(15)
    ax.barh(range(len(top_15)), top_15['Importance'], color=colors[idx], alpha=0.8)
    ax.set_yticks(range(len(top_15)))
    ax.set_yticklabels(top_15['Feature'], fontsize=9)
    ax.set_xlabel('Importance', fontsize=11, fontweight='bold')
    ax.set_title(f'{name}\nTop 15 Features', fontsize=12, fontweight='bold')
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=0.3)

    # Print top 5 for this model
    print(f"\n{name} - Top 5 Features:")
    print("-" * 80)
    print(importance_df.head(5).to_string(index=False))

plt.tight_layout()
plt.show()


In [ ]:
# Visualize Random Forest comparison - Confusion Matrices for all m values
model_names = list(rf_results.keys())
n_models = len(model_names)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Random Forest: Confusion Matrices for Different m Values',
             fontsize=16, fontweight='bold', y=1.02)

colors_maps = ['Blues', 'Greens', 'Oranges']

for idx, name in enumerate(model_names):
    ax = axes[idx]
    cm = rf_results[name]['confusion_matrix']

    # Create heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap=colors_maps[idx], ax=ax,
                cbar=False, annot_kws={'fontsize': 14, 'fontweight': 'bold'})

    # Labels and title
    ax.set_xlabel('Predicted', fontsize=12, fontweight='bold')
    ax.set_ylabel('Actual', fontsize=12, fontweight='bold')
    ax.set_title(f'{name}\nTest Error: {rf_results[name]["test_error"]:.4f} | AUC: {rf_results[name]["auc"]:.4f}',
                fontsize=12, fontweight='bold', pad=10)
    ax.set_xticklabels(['Inactive', 'Active'], fontsize=11)
    ax.set_yticklabels(['Inactive', 'Active'], fontsize=11, rotation=0)

    # Calculate and display accuracy metrics
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    # Add text box with metrics
    textstr = f'Accuracy: {accuracy:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}'
    props = dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='gray')
    ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=9,
            verticalalignment='top', bbox=props)

plt.show()
plt.tight_layout()

In [ ]:
# test for different number of trees
# Test Classification Error vs Number of Trees
print("="*80)
print("RANDOM FOREST: TEST ERROR vs NUMBER OF TREES")
print("="*80)

# Train models with increasing number of trees to track error progression
n_tree_values = [10, 25, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
error_progression = {name: [] for name in m_values.keys()}

print("\nTraining Random Forest models with varying tree counts...")
print("This may take a moment...\n")

for n_trees in n_tree_values:
    print(f"Training with {n_trees} trees...", end=' ')
    for name, m_val in m_values.items():
        # Train model with current number of trees
        rf_temp = RandomForestClassifier(
            n_estimators=n_trees,
            max_features=m_val,
            random_state=42,
            n_jobs=-1,
            min_samples_split=5,
            min_samples_leaf=2
        )
        rf_temp.fit(X_train, y_train)

        # Calculate test error
        y_pred_temp = rf_temp.predict(X_test)
        test_error = 1 - accuracy_score(y_test, y_pred_temp)
        error_progression[name].append(test_error)
    print("✓")

print("\nTraining complete!")

# Visualize error progression
fig, ax = plt.subplots(1, 1, figsize=(12, 7))

colors_line = ['#ff6b6b', '#4ecdc4', '#45b7d1']
markers = ['o', 's', 'D']

for idx, (name, errors) in enumerate(error_progression.items()):
    ax.plot(n_tree_values, errors,
            marker=markers[idx],
            linewidth=2.5,
            markersize=8,
            label=name,
            color=colors_line[idx],
            alpha=0.8)

    # Annotate final error value
    final_error = errors[-1]
    ax.annotate(f'{final_error:.4f}',
                xy=(n_tree_values[-1], final_error),
                xytext=(10, 0),
                textcoords='offset points',
                fontsize=9,
                fontweight='bold',
                color=colors_line[idx])

ax.set_xlabel('Number of Trees', fontsize=13, fontweight='bold')
ax.set_ylabel('Test Classification Error', fontsize=13, fontweight='bold')
ax.set_title('Random Forest: Test Error vs Number of Trees\nfor Different m Values',
             fontsize=15, fontweight='bold', pad=20)
ax.legend(fontsize=11, loc='upper right', framealpha=0.9)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xlim([0, 520])

# Add horizontal line at each model's final error
for idx, (name, errors) in enumerate(error_progression.items()):
    ax.axhline(y=errors[-1], color=colors_line[idx], linestyle=':', alpha=0.3, linewidth=1)

plt.tight_layout()
plt.show()

# Print summary table
print("\n" + "="*80)
print("TEST ERROR PROGRESSION SUMMARY")
print("="*80)
summary_data = []
for name, errors in error_progression.items():
    summary_data.append({
        'Model': name,
        'Error @10 trees': f'{errors[0]:.4f}',
        'Error @100 trees': f'{errors[3]:.4f}',
        'Error @500 trees': f'{errors[-1]:.4f}',
        'Improvement': f'{(errors[0] - errors[-1])*100:.2f}%'
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

print("\n" + "="*80)
print("KEY OBSERVATIONS:")
print("="*80)
print("- Error decreases as number of trees increases (diminishing returns)")
print("- Most improvement occurs in first 100-200 trees")
print("- All models stabilize around 300-400 trees")
print("- Different m values show different convergence patterns")
print("="*80)

## 6. Gradient Boosting with Different Tree Depths

In [ ]:
# Train Gradient Boosting models with different max_depth values
# Focusing on depths 1, 2
depths = [1, 2]
gb_results = {}

print("="*80)
print("TRAINING GRADIENT BOOSTING MODELS WITH DIFFERENT DEPTHS")
print("="*80)

for depth in depths:
    print(f"\n{'='*80}")
    print(f"Training Gradient Boosting with max_depth = {depth}")
    print(f"{'='*80}")

    # Train Gradient Boosting
    gb_model = GradientBoostingClassifier(
        n_estimators=500,
        max_depth=depth,
        learning_rate=0.1,
        random_state=42,
        subsample=0.8,
        min_samples_split=5,
        min_samples_leaf=2
    )

    gb_model.fit(X_train, y_train)

    # Predictions
    y_train_pred = gb_model.predict(X_train)
    y_test_pred = gb_model.predict(X_test)
    y_test_proba = gb_model.predict_proba(X_test)[:, 1]

    # Calculate metrics
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    train_error = 1 - train_accuracy
    test_error = 1 - test_accuracy
    auc_score = roc_auc_score(y_test, y_test_proba)

    # Store results
    gb_results[depth] = {
        'model': gb_model,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'train_error': train_error,
        'test_error': test_error,
        'auc': auc_score,
        'predictions': y_test_pred,
        'probabilities': y_test_proba,
        'confusion_matrix': confusion_matrix(y_test, y_test_pred)
    }

    print(f"\nResults:")
    print(f"  Training Accuracy: {train_accuracy:.4f}")
    print(f"  Training Error:    {train_error:.4f}")
    print(f"  Test Accuracy:     {test_accuracy:.4f}")
    print(f"  Test Error:        {test_error:.4f}")
    print(f"  AUC Score:         {auc_score:.4f}")

print(f"\n{'='*80}")
print("All Gradient Boosting models trained successfully!")
print(f"{'='*80}")

In [ ]:
# Compare Gradient Boosting models
gb_comparison_df = pd.DataFrame({
    'Max_Depth': list(gb_results.keys()),
    'Train_Accuracy': [gb_results[k]['train_accuracy'] for k in gb_results.keys()],
    'Test_Accuracy': [gb_results[k]['test_accuracy'] for k in gb_results.keys()],
    'Train_Error': [gb_results[k]['train_error'] for k in gb_results.keys()],
    'Test_Error': [gb_results[k]['test_error'] for k in gb_results.keys()],
    'AUC': [gb_results[k]['auc'] for k in gb_results.keys()]
})

print("="*80)
print("GRADIENT BOOSTING MODEL COMPARISON")
print("="*80)
print(gb_comparison_df.to_string(index=False))

# Find best model
best_depth = gb_comparison_df.loc[gb_comparison_df['Test_Error'].idxmin(), 'Max_Depth']
print(f"\n{'='*80}")
print(f"Best Gradient Boosting Model: max_depth = {best_depth}")
print(f"  Test Error: {gb_results[best_depth]['test_error']:.4f}")
print(f"  Test Accuracy: {gb_results[best_depth]['test_accuracy']:.4f}")
print(f"  AUC: {gb_results[best_depth]['auc']:.4f}")
print(f"{'='*80}")

In [ ]:
# Visualize Gradient Boosting comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Confusion Matrix for Depth = 1
ax = axes[0]
cm = gb_results[1]['confusion_matrix']
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, cbar=False,
            annot_kws={'fontsize': 14, 'fontweight': 'bold'})
ax.set_xlabel('Predicted', fontsize=12, fontweight='bold')
ax.set_ylabel('Actual', fontsize=12, fontweight='bold')
ax.set_title(f'Confusion Matrix: Depth = 1 (Stumps)\nTest Error: {gb_results[1]["test_error"]:.4f}',
             fontsize=14, fontweight='bold')
ax.set_xticklabels(['Inactive', 'Active'], fontsize=11)
ax.set_yticklabels(['Inactive', 'Active'], fontsize=11, rotation=0)

# Confusion Matrix for Depth = 2
ax = axes[1]
cm = gb_results[2]['confusion_matrix']
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', ax=ax, cbar=False,
            annot_kws={'fontsize': 14, 'fontweight': 'bold'})
ax.set_xlabel('Predicted', fontsize=12, fontweight='bold')
ax.set_ylabel('Actual', fontsize=12, fontweight='bold')
ax.set_title(f'Confusion Matrix: Depth = 2\nTest Error: {gb_results[2]["test_error"]:.4f}',
             fontsize=14, fontweight='bold')
ax.set_xticklabels(['Inactive', 'Active'], fontsize=11)
ax.set_yticklabels(['Inactive', 'Active'], fontsize=11, rotation=0)

plt.tight_layout()
plt.show()

### Gradient Boosting with Different Shrinkage Factors (Learning Rates)

Now let's explore the effect of different shrinkage factors on Gradient Boosting performance.
We'll use depth=2 (a good baseline) and vary the learning rate.

In [ ]:
# Train Gradient Boosting with different shrinkage factors (learning rates)
learning_rates = [0.01, 0.05, 0.1]
gb_shrinkage_results = {}

print("="*80)
print("GRADIENT BOOSTING WITH DIFFERENT SHRINKAGE FACTORS")
print("Using max_depth=2, varying learning rate")
print("="*80)

for lr in learning_rates:
    print(f"\n{'='*80}")
    print(f"Training Gradient Boosting with learning_rate = {lr}")
    print(f"{'='*80}")

    # Train Gradient Boosting , depth = 1
    gb_model = GradientBoostingClassifier(
        n_estimators=500,
        max_depth=1,
        learning_rate=lr,
        random_state=42,
        subsample=0.8,
        min_samples_split=5,
        min_samples_leaf=2
    )

    gb_model.fit(X_train, y_train)

    # Predictions
    y_train_pred = gb_model.predict(X_train)
    y_test_pred = gb_model.predict(X_test)
    y_test_proba = gb_model.predict_proba(X_test)[:, 1]

    # Calculate metrics
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    train_error = 1 - train_accuracy
    test_error = 1 - test_accuracy
    auc_score = roc_auc_score(y_test, y_test_proba)

    # Store results
    gb_shrinkage_results[lr] = {
        'model': gb_model,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'train_error': train_error,
        'test_error': test_error,
        'auc': auc_score,
        'predictions': y_test_pred,
        'probabilities': y_test_proba,
        'confusion_matrix': confusion_matrix(y_test, y_test_pred)
    }

    print(f"\nResults:")
    print(f"  Training Accuracy: {train_accuracy:.4f}")
    print(f"  Training Error:    {train_error:.4f}")
    print(f"  Test Accuracy:     {test_accuracy:.4f}")
    print(f"  Test Error:        {test_error:.4f}")
    print(f"  AUC Score:         {auc_score:.4f}")

print(f"\n{'='*80}")
print("All shrinkage factor models trained successfully!")
print(f"{'='*80}")

In [ ]:
# Compare shrinkage factor models
shrinkage_df = pd.DataFrame({
    'Learning_Rate': list(gb_shrinkage_results.keys()),
    'Train_Accuracy': [gb_shrinkage_results[k]['train_accuracy'] for k in gb_shrinkage_results.keys()],
    'Test_Accuracy': [gb_shrinkage_results[k]['test_accuracy'] for k in gb_shrinkage_results.keys()],
    'Train_Error': [gb_shrinkage_results[k]['train_error'] for k in gb_shrinkage_results.keys()],
    'Test_Error': [gb_shrinkage_results[k]['test_error'] for k in gb_shrinkage_results.keys()],
    'AUC': [gb_shrinkage_results[k]['auc'] for k in gb_shrinkage_results.keys()]
})

print("="*80)
print("GRADIENT BOOSTING: SHRINKAGE FACTOR COMPARISON")
print("="*80)
print(shrinkage_df.to_string(index=False))

# Find best learning rate
best_lr = shrinkage_df.loc[shrinkage_df['Test_Error'].idxmin(), 'Learning_Rate']
print(f"\n{'='*80}")
print(f"Best Learning Rate: {best_lr}")
print(f"  Test Error: {gb_shrinkage_results[best_lr]['test_error']:.4f}")
print(f"  Test Accuracy: {gb_shrinkage_results[best_lr]['test_accuracy']:.4f}")
print(f"  AUC: {gb_shrinkage_results[best_lr]['auc']:.4f}")
print(f"{'='*80}")

In [ ]:
# Visualize Shrinkage Factor Effect - Confusion Matrices for Different Learning Rates
n_rates = len(learning_rates)
fig, axes = plt.subplots(1, n_rates, figsize=(6*n_rates, 5))
fig.suptitle('Gradient Boosting: Confusion Matrices for Different Learning Rates\n(depth=2, 500 trees)',
             fontsize=16, fontweight='bold', y=1.02)

colors_maps = ['Blues', 'Greens', 'Oranges']

for idx, lr in enumerate(learning_rates):
    ax = axes[idx]
    cm = gb_shrinkage_results[lr]['confusion_matrix']

    # Create heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap=colors_maps[idx], ax=ax, cbar=False,
                annot_kws={'fontsize': 14, 'fontweight': 'bold'})

    # Labels and title
    ax.set_xlabel('Predicted', fontsize=12, fontweight='bold')
    ax.set_ylabel('Actual', fontsize=12, fontweight='bold')
    ax.set_title(f'Learning Rate = {lr}\nTest Error: {gb_shrinkage_results[lr]["test_error"]:.4f} | AUC: {gb_shrinkage_results[lr]["auc"]:.4f}',
                fontsize=13, fontweight='bold', pad=10)
    ax.set_xticklabels(['Inactive', 'Active'], fontsize=11)
    ax.set_yticklabels(['Inactive', 'Active'], fontsize=11, rotation=0)

    # Calculate and display accuracy metrics
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    # Add text box with metrics
    textstr = f'Accuracy: {accuracy:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}'
    props = dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='gray')
    ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=9,
            verticalalignment='top', bbox=props)

    # Highlight best model
    if lr == best_lr:
        for spine in ax.spines.values():
            spine.set_edgecolor('gold')
            spine.set_linewidth(3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("SHRINKAGE FACTOR ANALYSIS SUMMARY")
print("="*80)
print(f"Best Learning Rate: {best_lr}")
print(f"  - Lower learning rates (e.g., 0.01) require more trees but may generalize better")
print(f"  - Higher learning rates (e.g., 0.1+) converge faster but may overfit")
print(f"  - The optimal learning rate balances convergence speed and model performance")
print("="*80)

### Test Error vs Number of Trees Comparison

Compare how test classification error changes with the number of trees for different models:
- **GB depth=1, lr=0.1**
- **GB depth=2, lr=0.1**
- **GB depth=1, lr=0.01**
- **GB depth=1, lr=0.05**
- **RF m=sqrt(p)=5**

In [ ]:
# Train models with varying number of trees for comparison
n_trees_list = [10, 25, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
error_comparison = {}

print("="*80)
print("TRAINING MODELS WITH VARYING NUMBER OF TREES")
print("="*80)

# 1. Random Forest m = sqrt(p) = 5
print("\n" + "="*80)
print("Training Random Forest (m = sqrt(p) = 5) with varying trees...")
print("="*80)
rf_errors = []
for n_trees in n_trees_list:
    rf_temp = RandomForestClassifier(
        n_estimators=n_trees,
        max_features=int(np.sqrt(n_features)),
        random_state=42,
        n_jobs=-1,
        min_samples_split=5,
        min_samples_leaf=2
    )
    rf_temp.fit(X_train, y_train)
    y_pred = rf_temp.predict(X_test)
    test_error = 1 - accuracy_score(y_test, y_pred)
    rf_errors.append(test_error)
    print(f"  Trees: {n_trees:3d} | Test Error: {test_error:.4f}")

error_comparison['RF m=sqrt(p)=5'] = rf_errors

# 2. GB depth=1, lr=0.1
print("\n" + "="*80)
print("Training GB (depth=1, lr=0.1) with varying trees...")
print("="*80)
gb_d1_lr01_errors = []
for n_trees in n_trees_list:
    gb_temp = GradientBoostingClassifier(
        n_estimators=n_trees,
        max_depth=1,
        learning_rate=0.1,
        random_state=42,
        subsample=0.8,
        min_samples_split=5,
        min_samples_leaf=2
    )
    gb_temp.fit(X_train, y_train)
    y_pred = gb_temp.predict(X_test)
    test_error = 1 - accuracy_score(y_test, y_pred)
    gb_d1_lr01_errors.append(test_error)
    print(f"  Trees: {n_trees:3d} | Test Error: {test_error:.4f}")

error_comparison['GB depth=1, lr=0.1'] = gb_d1_lr01_errors

# 3. GB depth=2, lr=0.1
print("\n" + "="*80)
print("Training GB (depth=2, lr=0.1) with varying trees...")
print("="*80)
gb_d2_lr01_errors = []
for n_trees in n_trees_list:
    gb_temp = GradientBoostingClassifier(
        n_estimators=n_trees,
        max_depth=2,
        learning_rate=0.1,
        random_state=42,
        subsample=0.8,
        min_samples_split=5,
        min_samples_leaf=2
    )
    gb_temp.fit(X_train, y_train)
    y_pred = gb_temp.predict(X_test)
    test_error = 1 - accuracy_score(y_test, y_pred)
    gb_d2_lr01_errors.append(test_error)
    print(f"  Trees: {n_trees:3d} | Test Error: {test_error:.4f}")

error_comparison['GB depth=2, lr=0.1'] = gb_d2_lr01_errors

# 4. GB depth=1, lr=0.01
print("\n" + "="*80)
print("Training GB (depth=1, lr=0.01) with varying trees...")
print("="*80)
gb_d1_lr001_errors = []
for n_trees in n_trees_list:
    gb_temp = GradientBoostingClassifier(
        n_estimators=n_trees,
        max_depth=1,
        learning_rate=0.01,
        random_state=42,
        subsample=0.8,
        min_samples_split=5,
        min_samples_leaf=2
    )
    gb_temp.fit(X_train, y_train)
    y_pred = gb_temp.predict(X_test)
    test_error = 1 - accuracy_score(y_test, y_pred)
    gb_d1_lr001_errors.append(test_error)
    print(f"  Trees: {n_trees:3d} | Test Error: {test_error:.4f}")

error_comparison['GB depth=1, lr=0.01'] = gb_d1_lr001_errors

# 5. GB depth=1, lr=0.05
print("\n" + "="*80)
print("Training GB (depth=1, lr=0.05) with varying trees...")
print("="*80)
gb_d1_lr005_errors = []
for n_trees in n_trees_list:
    gb_temp = GradientBoostingClassifier(
        n_estimators=n_trees,
        max_depth=1,
        learning_rate=0.05,
        random_state=42,
        subsample=0.8,
        min_samples_split=5,
        min_samples_leaf=2
    )
    gb_temp.fit(X_train, y_train)
    y_pred = gb_temp.predict(X_test)
    test_error = 1 - accuracy_score(y_test, y_pred)
    gb_d1_lr005_errors.append(test_error)
    print(f"  Trees: {n_trees:3d} | Test Error: {test_error:.4f}")

error_comparison['GB depth=1, lr=0.05'] = gb_d1_lr005_errors

print("\n" + "="*80)
print("All models trained successfully!")
print("="*80)

In [ ]:
# Plot Test Error vs Number of Trees
plt.figure(figsize=(14, 8))

colors = ['steelblue', 'coral', 'forestgreen', 'purple', 'orange']
markers = ['o', 's', '^', 'D', 'v']
linestyles = ['-', '-', '-', '--', '--']

for idx, (model_name, errors) in enumerate(error_comparison.items()):
    plt.plot(n_trees_list, errors,
             marker=markers[idx],
             linewidth=2.5,
             markersize=8,
             color=colors[idx],
             linestyle=linestyles[idx],
             label=model_name,
             alpha=0.8)

plt.xlabel('Number of Trees', fontsize=14, fontweight='bold')
plt.ylabel('Test Classification Error', fontsize=14, fontweight='bold')
plt.title('Test Error vs Number of Trees: Model Comparison', fontsize=16, fontweight='bold')
plt.legend(fontsize=11, loc='upper right')
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*80)
print("SUMMARY: Final Test Errors at 500 Trees")
print("="*80)
for model_name, errors in error_comparison.items():
    final_error = errors[-1]
    initial_error = errors[0]
    improvement = (initial_error - final_error) * 100
    print(f"{model_name:30s} | Final Error: {final_error:.4f} | Improvement: {improvement:.2f}%")
print("="*80)

print("\n" + "="*80)
print("KEY INSIGHTS:")
print("="*80)
print("1. Learning Rate Effect (GB depth=1):")
print("   - lr=0.01: Slowest convergence, may need more trees")
print("   - lr=0.05: Moderate convergence")
print("   - lr=0.1:  Fastest convergence")
print("\n2. Depth Effect (GB with lr=0.1):")
print("   - depth=1 (stumps): Simple weak learners")
print("   - depth=2: Better performance, more complex trees")
print("\n3. Random Forest:")
print("   - Parallel training, different convergence pattern")
print("   - Performance stabilizes around 200-300 trees")
print("="*80)

### Focused Comparison: Gradient Boosting vs Random Forest

Let's compare specific models:
- **Gradient Boosting**: depths 1, 2, and 5
- **Random Forest**: m = 5 and m = sqrt(p)

## 7. Overall Model Comparison and Feature Importance

In [ ]:
# Compare all models (Random Forest and Gradient Boosting)
print("="*80)
print("FINAL MODEL COMPARISON: RANDOM FOREST VS GRADIENT BOOSTING")
print("="*80)

all_models = []

# Add Random Forest models
for name in rf_results.keys():
    all_models.append({
        'Model Type': 'Random Forest',
        'Configuration': name,
        'Test Error': rf_results[name]['test_error'],
        'Test Accuracy': rf_results[name]['test_accuracy'],
        'AUC': rf_results[name]['auc']
    })

# Add Gradient Boosting models (main depth variations)
for depth in gb_results.keys():
    all_models.append({
        'Model Type': 'Gradient Boosting',
        'Configuration': f'depth={depth}, lr=0.1',
        'Test Error': gb_results[depth]['test_error'],
        'Test Accuracy': gb_results[depth]['test_accuracy'],
        'AUC': gb_results[depth]['auc']
    })

# Add Gradient Boosting models with different learning rates (depth=1)
for lr in gb_shrinkage_results.keys():
    all_models.append({
        'Model Type': 'Gradient Boosting',
        'Configuration': f'depth=1, lr={lr}',
        'Test Error': gb_shrinkage_results[lr]['test_error'],
        'Test Accuracy': gb_shrinkage_results[lr]['test_accuracy'],
        'AUC': gb_shrinkage_results[lr]['auc']
    })

final_comparison_df = pd.DataFrame(all_models)
final_comparison_df = final_comparison_df.sort_values('Test Error')

print(final_comparison_df.to_string(index=False))

# Find overall best model
best_idx = final_comparison_df['Test Error'].idxmin()
best_overall = final_comparison_df.loc[best_idx]

print(f"\n{'='*80}")
print(f"OVERALL BEST MODEL:")
print(f"{'='*80}")
print(f"Model Type:     {best_overall['Model Type']}")
print(f"Configuration:  {best_overall['Configuration']}")
print(f"Test Error:     {best_overall['Test Error']:.4f}")
print(f"Test Accuracy:  {best_overall['Test Accuracy']:.4f}")
print(f"AUC:            {best_overall['AUC']:.4f}")
print(f"{'='*80}")

In [ ]:
# Feature Importance Analysis
print("="*80)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*80)

# Get feature importance from best Random Forest model
best_rf_model = rf_results[best_model_name]['model']
feature_importance_rf = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

# Get feature importance from best Gradient Boosting model
best_gb_model = gb_results[best_depth]['model']
feature_importance_gb = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_gb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nTop 15 Most Important Features (Random Forest - {best_model_name}):")
print("="*80)
print(feature_importance_rf.head(15).to_string(index=False))

print(f"\n\nTop 15 Most Important Features (Gradient Boosting - depth={best_depth}):")
print("="*80)
print(feature_importance_gb.head(15).to_string(index=False))

In [ ]:
# Bagging Model (m = p) - Detailed Feature Importance Analysis
print("="*80)
print("BAGGING MODEL FEATURE IMPORTANCE (Gini Impurity Reduction)")
print("="*80)

# Get bagging model (m = p)
bagging_model = rf_results['m = p (Bagging)']['model']
bagging_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': bagging_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nMethod: Mean Decrease in Impurity (MDI) - Gini Reduction")
print(f"Description: Total reduction in Gini impurity across all {bagging_model.n_estimators} trees")
print(f"              weighted by the probability of reaching each node\n")

print("Top 20 Most Influential Features:")
print("-" * 80)
print(bagging_importance.head(20).to_string(index=False))

# Visualize relative influence for bagging model
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top 20 features
ax = axes[0]
top_20 = bagging_importance.head(20)
ax.barh(range(len(top_20)), top_20['Importance'], color='#45b7d1', alpha=0.8, edgecolor='black')
ax.set_yticks(range(len(top_20)))
ax.set_yticklabels(top_20['Feature'], fontsize=10)
ax.set_xlabel('Relative Influence (Gini Reduction)', fontsize=12, fontweight='bold')
ax.set_title('Bagging Model: Top 20 Feature Importance\n(m = p, using all features)',
             fontsize=13, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

# Add percentage labels
for i, (idx, row) in enumerate(top_20.iterrows()):
    pct = row['Importance'] / bagging_importance['Importance'].sum() * 100
    ax.text(row['Importance'], i, f' {pct:.1f}%', va='center', fontsize=8)

# Cumulative importance
ax = axes[1]
cumsum = bagging_importance['Importance'].cumsum()
cumsum_pct = (cumsum / bagging_importance['Importance'].sum() * 100)
ax.plot(range(1, len(cumsum_pct) + 1), cumsum_pct, linewidth=2.5, color='#45b7d1')
ax.axhline(y=80, color='red', linestyle='--', linewidth=2, label='80% threshold')
ax.axhline(y=90, color='orange', linestyle='--', linewidth=2, label='90% threshold')
ax.set_xlabel('Number of Features', fontsize=12, fontweight='bold')
ax.set_ylabel('Cumulative Importance (%)', fontsize=12, fontweight='bold')
ax.set_title('Bagging Model: Cumulative Feature Importance', fontsize=13, fontweight='bold')
ax.grid(alpha=0.3)
ax.legend()
ax.set_ylim([0, 105])

# Find features needed for 80% and 90%
features_80 = (cumsum_pct >= 80).argmax() + 1
features_90 = (cumsum_pct >= 90).argmax() + 1
ax.axvline(x=features_80, color='red', linestyle=':', alpha=0.5)
ax.axvline(x=features_90, color='orange', linestyle=':', alpha=0.5)
ax.text(features_80, 5, f'{features_80} features', rotation=90, va='bottom', ha='right', fontsize=9)
ax.text(features_90, 5, f'{features_90} features', rotation=90, va='bottom', ha='right', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\n{'='*80}")
print(f"FEATURE IMPORTANCE SUMMARY:")
print(f"{'='*80}")
print(f"Top 5 features explain: {cumsum_pct.iloc[4]:.1f}% of total importance")
print(f"Top 10 features explain: {cumsum_pct.iloc[9]:.1f}% of total importance")
print(f"Top 20 features explain: {cumsum_pct.iloc[19]:.1f}% of total importance")
print(f"{features_80} features needed for 80% cumulative importance")
print(f"{features_90} features needed for 90% cumulative importance")
print(f"{'='*80}")

## 8. Key Findings and Conclusions

## Comprehensive Summary of Key Findings

### 1. Dataset Overview
- **Total Records**: Netfix simulate data
- **Features**: 30 features after engineering (demographics, subscription, engagement, content)
- **Target Distribution**: Binary classification (active vs inactive users)
- **Train/Test Split**: 80/20 with stratification

### 2. Random Forest Performance (500 trees)

**Three configurations tested:**
- **m = sqrt(p) ≈ 5**: Default classification setting
- **m = p/2 = 15**: Moderate feature selection
- **m = p = 30 (Bagging)**: Uses all features

**Key Findings:**
- All three RF models showed strong performance (test error typically < 5%)
- **m = sqrt(p)** provides good balance between variance reduction and tree diversity
- **m = p/2** offers middle ground with reasonable decorrelation
- **m = p (Bagging)** uses all features but may have higher tree correlation
- Models converge around 200-300 trees, with diminishing returns after 400 trees

### 3. Gradient Boosting Performance

**Depth Analysis (learning rate = 0.1, 500 trees):**
- **Depth = 1 (Stumps)**: Simplest weak learners, may underfit
- **Depth = 2**: Good balance between complexity and generalization

**Key Findings:**
- Sequential learning allows GB to correct previous errors
- Depth = 2 often provides optimal bias-variance tradeoff
- Deeper trees (depth = 5) achieve lower training error but need careful monitoring

### 4. Shrinkage Factor (Learning Rate) Analysis

**Tested learning rates (depth = 1, 500 trees):**
- **lr = 0.01**: Very slow learning, requires many trees, most conservative
- **lr = 0.05**: Moderate convergence speed, balanced approach
- **lr = 0.1**: Faster convergence, standard default value

**Key Findings:**
- **Lower learning rates** (0.01-0.05):
  - Slower convergence but potentially better generalization
  - Require more trees to achieve optimal performance
  - More resistant to overfitting
  
- **Higher learning rates** (0.1+):
  - Faster convergence (fewer trees needed)
  - Risk of overshooting optimal solution
  - May overfit with too many trees

- **Optimal strategy**: Balance between learning rate and number of trees
  - lr=0.1 with 300-500 trees often works well
  - lr=0.01 may need 1000+ trees for full convergence

### 5. Random Forest vs Gradient Boosting

**Random Forest Advantages:**
- Parallel training (faster with multiple cores)
- Less sensitive to hyperparameters
- Built-in feature importance via Gini reduction
- Naturally handles overfitting through averaging
- More stable predictions (less variance)

**Gradient Boosting Advantages:**
- Sequential error correction (often higher accuracy)
- More flexible with depth and learning rate tuning
- Can achieve better performance with proper tuning
- Better at capturing complex interactions

**Performance Comparison:**
- GB with optimal settings typically edges out RF in test accuracy
- RF provides more consistent results across different m values
- GB requires more careful hyperparameter tuning

### 6. Convergence Analysis: Test Error vs Number of Trees

**Models compared across 10-500 trees:**
1. RF m=sqrt(p)=5
2. GB depth=1, lr=0.1
3. GB depth=2, lr=0.1
4. GB depth=1, lr=0.01
5. GB depth=1, lr=0.05

**Key Findings:**
- **GB with lr=0.1**: Fastest convergence (stable by 100-150 trees)
- **GB with lr=0.05**: Moderate convergence (stable by 200-250 trees)
- **GB with lr=0.01**: Slowest convergence (may need 500+ trees)
- **RF m=sqrt(p)**: Steady convergence, stable by 250-300 trees
- **GB depth=2 vs depth=1**: Deeper trees achieve lower error faster

**Practical Implications:**
- For quick prototyping: Use GB with lr=0.1 and 100-200 trees
- For production: Use lr=0.05 with 300-500 trees for better generalization
- RF offers good "out-of-the-box" performance without extensive tuning

### 7. Feature Importance Insights

**Top predictive features consistently included:**
- User engagement metrics (watch time, session count)
- Content interaction (recommendations clicked, searches)
- Subscription characteristics (plan type, monthly spend)
- Temporal patterns (subscription duration, recency)
- Demographics (age, household composition)

**Model-specific insights:**
- RF: More democratic feature importance (flatter distribution)
- GB: More concentrated importance on top features (steeper distribution)
- Both methods agree on top 5-10 most important features

### 8. Model Selection Recommendations

**Choose Random Forest when:**
- Need fast, parallel training
- Want stable, interpretable results
- Have limited time for hyperparameter tuning
- Prefer robust performance across datasets

**Choose Gradient Boosting when:**
- Need maximum predictive accuracy
- Have time for careful hyperparameter tuning
- Can afford sequential training time
- Want to fine-tune bias-variance tradeoff

**Hyperparameter Guidelines:**
- **RF**: Start with m=sqrt(p), 300-500 trees
- **GB**: Start with depth=2, lr=0.1, 200-300 trees
- **Fine-tuning**: Adjust based on validation error curves

### 9. Business Implications for Netflix

**Churn Prediction:**
- Models achieve high accuracy (>95%) in predicting user activity status
- Early warning system possible for users at risk of becoming inactive
- Key intervention points identified through feature importance

**Actionable Insights:**
1. **Engagement monitoring**: Track watch time and session frequency
2. **Recommendation optimization**: Improve click-through rates
3. **Personalization**: Target content based on viewing patterns
4. **Retention campaigns**: Focus on users with declining engagement metrics
5. **Subscription optimization**: Address plan-specific churn patterns

### 10. Technical Conclusions

**Model Performance Summary:**
- Both RF and GB achieve excellent performance (test error 3-5%)
- Ensemble methods vastly outperform single decision trees
- Proper hyperparameter tuning yields 1-2% improvement
- Learning rate is critical for GB convergence and generalization

**Best Practices Identified:**
1. Always validate on separate test set
2. Monitor train vs test error to detect overfitting
3. Use cross-validation for final model selection
4. Consider computational constraints when choosing method
5. Balance model complexity with interpretability needs

**Final Model Recommendation:**
- **For deployment**: GB with depth=2, lr=0.1, 300 trees
- **For interpretability**: RF with m=sqrt(p), 400 trees
- **For experimentation**: Test both and ensemble predictions